In [1]:
import numpy as np
import itertools
import pandas as pd
import json

from sklearn.isotonic import IsotonicRegression
from sklearn.calibration import calibration_curve, _SigmoidCalibration
from ampligraph.evaluation import evaluate_performance, mr_score, mrr_score, hits_at_n_score, generate_corruptions_for_eval
from sklearn.metrics import brier_score_loss, log_loss, accuracy_score
from scipy.special import expit

from ampligraph.datasets import load_wordnet11
from ampligraph.latent_features.models import TransE, ComplEx, DistMult

In [2]:
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_VISIBLE_DEVICES=1


In [3]:
X_train = (pd.read_csv("TransC/data/YAGO39K/Train/triple2id.txt", sep=' ', skiprows=1, names=['s', 'o', 'p'])
           [['s', 'p', 'o']]
           .reindex()
           .values)

In [4]:
ent = np.unique(np.concatenate((X_train[:, 0], X_train[:, 2])))

In [5]:
X_valid_pos = (pd.read_csv("TransC/data/YAGO39K/Valid/triple2id_positive.txt", sep=' ', skiprows=1, names=['s', 'o', 'p'])
               [['s', 'p', 'o']]
               .reindex()
               .query("s in @ent and o in @ent")
               .values)

X_valid_neg = (pd.read_csv("TransC/data/YAGO39K/Valid/triple2id_negative.txt", sep=' ', skiprows=1, names=['s', 'o', 'p'])
               [['s', 'p', 'o']]
               .reindex()
               .query("s in @ent and o in @ent")
               .values)

In [6]:
X_test_pos = (pd.read_csv("TransC/data/YAGO39K/Valid/triple2id_positive.txt", sep=' ', skiprows=1, names=['s', 'o', 'p'])
              [['s', 'p', 'o']]
              .reindex()
              .query("s in @ent and o in @ent")
              .values)

X_test_neg = (pd.read_csv("TransC/data/YAGO39K/Valid/triple2id_negative.txt", sep=' ', skiprows=1, names=['s', 'o', 'p'])
              [['s', 'p', 'o']]
              .query("s in @ent and o in @ent")
              .reindex()
              .values)

In [7]:
X = {
    'train': X_train,
    'valid': np.concatenate((X_valid_pos, X_valid_neg)),
    'test': np.concatenate((X_test_pos, X_test_neg)),
    'valid_labels': np.concatenate((np.full(len(X_valid_pos), '1'), np.full(len(X_valid_neg), '-1'))),
    'test_labels': np.concatenate((np.full(len(X_test_pos), '1'), np.full(len(X_test_neg), '-1')))
}

In [8]:
import types
from functools import partial
import tensorflow as tf
from sklearn.utils import check_random_state
from tqdm import tqdm
from ampligraph.datasets import AmpligraphDatasetAdapter, NumpyDatasetAdapter
from ampligraph.evaluation import generate_corruptions_for_fit, to_idx, generate_corruptions_for_eval, \
    hits_at_n_score, mrr_score


def generate_corruptions(self, X_pos, batches_count, epochs):
    try:
        tf.reset_default_graph()
        self.rnd = check_random_state(self.seed)
        tf.random.set_random_seed(self.seed)

        self._load_model_from_trained_params()

        dataset_handle = NumpyDatasetAdapter()
        dataset_handle.use_mappings(self.rel_to_idx, self.ent_to_idx)

        dataset_handle.set_data(X_pos, "pos")

        batch_size_pos = int(np.ceil(dataset_handle.get_size("pos") / batches_count))

        gen_fn = partial(dataset_handle.get_next_train_batch, batch_size=batch_size_pos, dataset_type="pos")
        dataset = tf.data.Dataset.from_generator(gen_fn,
                                                 output_types=tf.int32,
                                                 output_shapes=(None, 3))
        dataset = dataset.repeat().prefetch(1)
        dataset_iter = tf.data.make_one_shot_iterator(dataset)

        x_pos_tf = dataset_iter.get_next()

        e_s, e_p, e_o = self._lookup_embeddings(x_pos_tf)
        scores_pos = self._fn(e_s, e_p, e_o)

        x_neg_tf = generate_corruptions_for_fit(x_pos_tf,
                                                entities_list=None,
                                                eta=1,
                                                corrupt_side='s+o',
                                                entities_size=0,
                                                rnd=self.seed)

        e_s_neg, e_p_neg, e_o_neg = self._lookup_embeddings(x_neg_tf)
        scores = self._fn(e_s_neg, e_p_neg, e_o_neg)

        epoch_iterator_with_progress = tqdm(range(1, epochs + 1), disable=(not self.verbose), unit='epoch')

        scores_list = []
        with tf.Session(config=self.tf_config) as sess:
            sess.run(tf.global_variables_initializer())
            for _ in epoch_iterator_with_progress:
                losses = []
                for batch in range(batches_count):
                    scores_list.append(sess.run(scores))

        dataset_handle.cleanup()
        return np.concatenate(scores_list)
    
    except Exception as e:
        dataset_handle.cleanup()
        raise e

In [9]:
def pos_iso(cal_model, pos_scores, neg_scores, positive_base_rate):
    weigths_pos = len(neg_scores) / len(pos_scores)
    weights_neg = (1.0 - positive_base_rate) / positive_base_rate
    weights = np.concatenate((np.full(pos_scores.shape, weigths_pos),
                              np.full(neg_scores.shape, weights_neg))).astype(float)
    target =  np.concatenate((np.ones(pos_scores.shape), np.zeros(neg_scores.shape))).astype(float)
    x = np.concatenate((pos_scores, neg_scores)).astype(float)
    
    cal_model.fit(x, target, sample_weight=weights)
    return cal_model

In [ ]:
losses =  ['self_adversarial', 'pairwise', 'nll', 'multiclass_nll']
models = [TransE, DistMult, ComplEx]

results = []

for m, l in itertools.product(models, losses):
    model = m(batches_count=64, seed=0, epochs=1000, k=100, eta=20,
                   optimizer='adam', optimizer_params={'lr':0.0001},
                   loss=l, verbose=False)

    model.fit(X['train'])
    
    scores = model.predict(X['test'])

    model.calibrate(X_valid_pos, batches_count=10, epochs=1000, positive_base_rate=0.5)
    print("pos", model.calibration_parameters)
    probas1 = model.predict_proba(X['test'])

    model.calibrate(X_valid_pos, X_valid_neg)
    print("pos neg", model.calibration_parameters)
    probas2 = model.predict_proba(X['test'])
    
    val_scores = model.predict(X['valid'])
    ir = IsotonicRegression(out_of_bounds='clip')
    ir.fit(np.squeeze(val_scores).astype(float), (X['valid_labels'] == "1").astype(float))
    probas3 = ir.predict(np.squeeze(scores).astype(float))
    
    model.generate_corruptions = types.MethodType(generate_corruptions, model)
    corruptions = model.generate_corruptions(X_valid_pos, batches_count=10, epochs=1000)
    val_pos_scores = np.squeeze(model.predict(X_valid_pos))
    iso_pos = pos_iso(IsotonicRegression(out_of_bounds='clip'), val_pos_scores, corruptions, positive_base_rate=0.5)
    probas4 = iso_pos.predict(np.squeeze(scores).astype(float))

    sc_pos = pos_iso(_SigmoidCalibration(), val_pos_scores, corruptions, positive_base_rate=0.5)
    print("pos sc", sc_pos.a_, sc_pos.b_)
    probas5 = sc_pos.predict(np.squeeze(scores).astype(float))
    
    val_neg_scores = np.squeeze(model.predict(X_valid_neg))
    sc_pos_neg = pos_iso(_SigmoidCalibration(), val_pos_scores, val_neg_scores, positive_base_rate=0.5)
    print("pos neg sc", sc_pos_neg.a_, sc_pos_neg.b_)
    probas6 = sc_pos_neg.predict(np.squeeze(scores).astype(float))
    
    thresholds = {r: np.median(np.sort(val_scores[X['valid'][:, 1] == r])) for r in np.unique(X['valid'][:, 1])}
    thresholds_test = np.vectorize(thresholds.get)(X['test'][:, 1])
    per_relation_acc = accuracy_score(X['test_labels'] == "1", scores > thresholds_test)

    acc_uncalib = accuracy_score(X['test_labels'] == "1", expit(scores) > 0.5)

    acc1 = accuracy_score(X['test_labels'] == "1", probas1 > 0.5)
    acc2 = accuracy_score(X['test_labels'] == "1", probas2 > 0.5)
    acc3 = accuracy_score(X['test_labels'] == "1", probas3 > 0.5)
    acc4 = accuracy_score(X['test_labels'] == "1", probas4 > 0.5)
    acc5 = accuracy_score(X['test_labels'] == "1", probas5 > 0.5)
    acc6 = accuracy_score(X['test_labels'] == "1", probas6 > 0.5)
    
    filter_triples = np.concatenate((X['train'], X_valid_pos, X_test_pos))
    ranks = evaluate_performance(X_test_pos, 
                                 model=model, 
                                 filter_triples=filter_triples,
                                 use_default_protocol=True, 
                                 verbose=False)

    results.append({
        'model': m.__name__,
        'loss': l,
        'brier_score_scores': brier_score_loss(X['test_labels'] == "1", expit(scores)),
        'log_loss_scores': log_loss(X['test_labels'] == "1", expit(scores), eps=1e-7),
        'brier_score_probas_pos': brier_score_loss(X['test_labels'] == "1", probas1),
        'log_loss_probas_pos': log_loss(X['test_labels'] == "1", probas1, eps=1e-7),
        'brier_score_probas_pos_neg': brier_score_loss(X['test_labels'] == "1", probas2),
        'log_loss_probas_pos_neg': log_loss(X['test_labels'] == "1", probas2, eps=1e-7),
        'brier_score_probas_pos_neg_iso': brier_score_loss(X['test_labels'] == "1", probas3),
        'log_loss_probas_pos_neg_iso': log_loss(X['test_labels'] == "1", probas3, eps=1e-7),
        'brier_score_probas_pos_iso': brier_score_loss(X['test_labels'] == "1", probas4),
        'log_loss_probas_pos_iso': log_loss(X['test_labels'] == "1", probas4, eps=1e-7),
        'brier_score_probas_pos_sc': brier_score_loss(X['test_labels'] == "1", probas5),
        'log_loss_probas_pos_sc': log_loss(X['test_labels'] == "1", probas5, eps=1e-7),
        'brier_score_probas_pos_neg_sc': brier_score_loss(X['test_labels'] == "1", probas6),
        'log_loss_probas_pos_neg_sc': log_loss(X['test_labels'] == "1", probas6, eps=1e-7),
        'metrics_mrr': mrr_score(ranks), 
        'metrics_hits@10': hits_at_n_score(ranks, n=10),
        'metrics_mr': mr_score(ranks),
        'accuracy_per_relation': per_relation_acc,
        'accuracy_uncalib': acc_uncalib,
        'accuracy_pos': acc1,
        'accuracy_pos_neg': acc2,
        'accuracy_pos_neg_iso': acc3,
        'accuracy_pos_iso': acc4,
        'accuracy_pos_sc': acc5,
        'accuracy_pos_neg_sc': acc6
    })
        
    print(json.dumps(results[-1], indent=2))

WARNING - From /home/ptabacof/.conda/envs/tf_gpu/lib/python3.7/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.
WARNING - From /home/ptabacof/.conda/envs/tf_gpu/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py:429: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    
WARNING - From /home/ptabacof/.

In [19]:
import pandas as pd

In [20]:
def highlight_min(s):
    is_min = s == s.min()
    return ['font-weight: bold' if v else '' for v in is_min]

In [21]:
df = pd.DataFrame(results).set_index(['model', 'loss'])

In [22]:
bs = df[(c for c in df.columns if c.startswith('brier'))]
bs.columns = [c[len("brier_score_"):] for c in bs.columns]
bs.style.apply(highlight_min, axis=1)

In [23]:
ll = df[(c for c in df.columns if c.startswith('log_loss'))]
ll.columns = [c[len("log_loss_"):] for c in ll.columns]
ll.style.apply(highlight_min, axis=1)

In [24]:
print((bs.reset_index()
 .query("loss == 'self_adversarial' ")
 [['model', 'scores', 'probas_pos_neg', 'probas_pos_neg_iso', 'probas_pos', 'probas_pos_iso']]
 .reset_index(drop=True)
 .round(3)
 .to_latex()))

\begin{tabular}{llrrrrr}
\toprule
{} &     model &  scores &  probas\_pos\_neg &  probas\_pos\_neg\_iso &  probas\_pos &  probas\_pos\_iso \\
\midrule
0 &    TransE &   0.363 &           0.095 &               0.093 &       0.106 &           0.110 \\
1 &  DistMult &   0.284 &           0.081 &               0.079 &       0.093 &           0.089 \\
2 &   ComplEx &   0.264 &           0.089 &               0.084 &       0.097 &           0.095 \\
\bottomrule
\end{tabular}



In [25]:
print(ll.reset_index()
 .query("loss == 'self_adversarial' ")
 [['model', 'scores', 'probas_pos_neg', 'probas_pos_neg_iso', 'probas_pos', 'probas_pos_iso']]
 .reset_index(drop=True)
  .round(3)
 .to_latex())

\begin{tabular}{llrrrrr}
\toprule
{} &     model &  scores &  probas\_pos\_neg &  probas\_pos\_neg\_iso &  probas\_pos &  probas\_pos\_iso \\
\midrule
0 &    TransE &   1.062 &           0.319 &               0.309 &       0.370 &           0.376 \\
1 &  DistMult &   1.043 &           0.279 &               0.266 &       0.311 &           0.308 \\
2 &   ComplEx &   1.199 &           0.305 &               0.278 &       0.323 &           0.313 \\
\bottomrule
\end{tabular}



In [33]:
print((acc*100).reset_index()
 .query("loss == 'self_adversarial' ")
 [['model', 'pos_neg', 'pos_neg_iso', 'pos', 'pos_iso',  'uncalib', 'per_relation']]
 .reset_index(drop=True)
  .round(1)
 .to_latex())

\begin{tabular}{llrrrrrr}
\toprule
{} &     model &  pos\_neg &  pos\_neg\_iso &   pos &  pos\_iso &  uncalib &  per\_relation \\
\midrule
0 &    TransE &     87.1 &         87.8 &  86.6 &     84.9 &     50.2 &          88.8 \\
1 &  DistMult &     88.9 &         89.3 &  87.7 &     88.5 &     56.7 &          90.2 \\
2 &   ComplEx &     87.3 &         88.2 &  86.6 &     87.2 &     61.1 &          89.4 \\
\bottomrule
\end{tabular}



In [27]:
metrics = df[(c for c in df.columns if c.startswith('metrics'))]
metrics.columns = [c[len("metrics_"):] for c in metrics.columns]
metrics

hits@10           mr       mrr
model    loss                                             
TransE   self_adversarial  0.319041   244.604493  0.168512
         pairwise          0.648666   500.205260  0.371433
         nll               0.112647  1014.709742  0.063875
         multiclass_nll    0.588346   216.314559  0.323279
DistMult self_adversarial  0.619883   635.099201  0.305570
         pairwise          0.631116   648.168485  0.337314
         nll               0.647262   744.198240  0.330295
         multiclass_nll    0.585268   397.900475  0.308723
ComplEx  self_adversarial  0.752673  1074.095853  0.530927
         pairwise          0.729776   586.873528  0.476573
         nll               0.731882   855.117021  0.493711
         multiclass_nll    0.747165   326.583702  0.475104

In [28]:
def highlight_max(s):
    is_min = s == s.max()
    return ['font-weight: bold' if v else '' for v in is_min]

acc = df[(c for c in df.columns if c.startswith('accuracy'))]
acc.columns = [c[len("accuracy_"):] for c in acc.columns]
acc.style.apply(highlight_max, axis=1)

In [30]:
df.corr(method='spearman')

,accuracy_per_relation,accuracy_pos,accuracy_pos_iso,accuracy_pos_neg,accuracy_pos_neg_iso,accuracy_pos_neg_sc,accuracy_pos_sc,accuracy_uncalib,brier_score_probas_pos,brier_score_probas_pos_iso,...,log_loss_probas_pos,log_loss_probas_pos_iso,log_loss_probas_pos_neg,log_loss_probas_pos_neg_iso,log_loss_probas_pos_neg_sc,log_loss_probas_pos_sc,log_loss_scores,metrics_hits@10,metrics_mr,metrics_mrr
accuracy_per_relation,1.000000,0.398601,0.447552,0.405594,0.384615,0.405594,0.419580,0.555248,-0.496503,-0.342657,...,-0.489510,-0.454545,-0.510490,-0.475524,-0.510490,-0.454545,0.069930,0.370629,-0.160839,0.342657
accuracy_pos,0.398601,1.000000,0.965035,0.993007,0.986014,0.993007,0.993007,0.619315,-0.930070,-0.986014,...,-0.895105,-0.818182,-0.888112,-0.937063,-0.888112,-0.832168,-0.293706,0.391608,0.174825,0.300699
accuracy_pos_iso,0.447552,0.965035,1.000000,0.958042,0.979021,0.958042,0.972028,0.612197,-0.839161,-0.965035,...,-0.804196,-0.797203,-0.811189,-0.888112,-0.811189,-0.741259,-0.195804,0.454545,0.188811,0.363636
accuracy_pos_neg,0.405594,0.993007,0.958042,1.000000,0.979021,1.000000,0.986014,0.626434,-0.923077,-0.979021,...,-0.888112,-0.811189,-0.881119,-0.930070,-0.881119,-0.825175,-0.300699,0.370629,0.167832,0.265734
accuracy_pos_neg_iso,0.384615,0.986014,0.979021,0.979021,1.000000,0.979021,0.993007,0.612197,-0.888112,-0.986014,...,-0.867133,-0.832168,-0.874126,-0.937063,-0.874126,-0.818182,-0.286713,0.398601,0.174825,0.300699
accuracy_pos_neg_sc,0.405594,0.993007,0.958042,1.000000,0.979021,1.000000,0.986014,0.626434,-0.923077,-0.979021,...,-0.888112,-0.811189,-0.881119,-0.930070,-0.881119,-0.825175,-0.300699,0.370629,0.167832,0.265734
accuracy_pos_sc,0.419580,0.993007,0.972028,0.986014,0.993007,0.986014,1.000000,0.612197,-0.923077,-0.979021,...,-0.888112,-0.811189,-0.895105,-0.944056,-0.895105,-0.825175,-0.279720,0.370629,0.146853,0.279720
accuracy_uncalib,0.555248,0.619315,0.612197,0.626434,0.612197,0.626434,0.612197,1.000000,-0.733212,-0.626434,...,-0.804398,-0.882702,-0.797279,-0.754568,-0.797279,-0.847109,-0.444910,0.637112,0.558807,0.555248
brier_score_probas_pos,-0.496503,-0.930070,-0.839161,-0.923077,-0.888112,-0.923077,-0.923077,-0.733212,1.000000,0.888112,...,0.979021,0.832168,0.972028,0.951049,0.972028,0.930070,0.370629,-0.370629,-0.181818,-0.286713
brier_score_probas_pos_iso,-0.342657,-0.986014,-0.965035,-0.979021,-0.986014,-0.979021,-0.979021,-0.626434,0.888112,1.000000,...,0.867133,0.846154,0.860140,0.923077,0.860140,0.818182,0.321678,-0.405594,-0.223776,-0.328671


In [34]:
len(np.unique(X['valid'][:, 1]))

33